In [ ]:
import geopandas as gpd

In [ ]:
# 参考 https://github.com/CNFeffery/DataScienceStudyNotes
#     （数据科学学习手札79）基于geopandas的空间数据分析——深入浅出分层设色

map_cn = gpd.read_file('zip://../map_of_China/china-shapefiles.zip!china-shapefiles',
                       layer='china',
                       encoding='utf-8')

# 由于每行数据是单独的面，因此按照其省份列OWNER融合
map_cn = map_cn.dissolve(by='OWNER').reset_index(drop=False)

map_jiuduanxian = gpd.read_file('zip://../map_of_China/china-shapefiles.zip!china-shapefiles',
                           layer='china_nine_dotted_line',
                           encoding='utf-8')

In [ ]:
map_cn.head()

In [ ]:
map_jiuduanxian.head()

In [ ]:
map_cn.query('not FCNAME.isnull()')

In [ ]:
sheng_map = dict()
for row in map_cn.iterrows():
    sheng_name = row[1]['OWNER']
    if type(sheng_name) is str:  # 有省名称的数据行
        sheng_map[sheng_name] = row[1]
        
print('\n'.join(sheng_map.keys()))
print(len(sheng_map))

In [ ]:
import pandas as pd

In [ ]:
# https://github.com/CSSEGISandData/COVID-19
covid_19_confirmed = pd.read_csv('../COVID-19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
covid_19_confirmed.head()

In [ ]:
covid_19_confirmed.query('`Country/Region`=="China"')

In [ ]:
covid_19_confirmed.head()

covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Province/State']='Taiwan'
covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Country/Region']='China'

covid_19_confirmed[covid_19_confirmed['Country/Region'] == 'China']

In [ ]:
sheng_en_to_cn = {
    'Beijing': '北京市',
    'Tianjin': '天津市',
    'Hebei': '河北省',
    'Shanxi': '山西省',
    'Inner Mongolia': '内蒙古自治区',
    'Liaoning': '辽宁省',
    'Jilin': '吉林省',
    'Heilongjiang': '黑龙江省',
    'Shanghai': '上海市',
    'Jiangsu': '江苏省',
    'Zhejiang': '浙江省',
    'Anhui': '安徽省',
    'Fujian': '福建省',
    'Jiangxi': '江西省',
    'Shandong': '山东省',
    'Henan': '河南省',
    'Hubei': '湖北省',
    'Hunan': '湖南省',
    'Guangdong': '广东省',
    'Guangxi': '广西壮族自治区',
    'Hainan': '海南省',
    'Chongqing': '重庆市',
    'Sichuan': '四川省',
    'Guizhou': '贵州省',
    'Yunnan': '云南省',
    'Tibet': '西藏自治区',
    'Shaanxi': '陕西省',
    'Gansu': '甘肃省',
    'Qinghai': '青海省',
    'Ningxia': '宁夏回族自治区',
    'Xinjiang': '新疆维吾尔自治区',
    'Taiwan': '台湾省',
    'Hong Kong': '香港特别行政区',
    'Macau': '澳门特别行政区'
}

assert(len(sheng_en_to_cn) == len(sheng_map))
assert(len(sheng_en_to_cn) == 34)
for k in sheng_en_to_cn:
    assert(sheng_en_to_cn[k] in sheng_map)

In [ ]:
covid_19_confirmed_China = covid_19_confirmed[covid_19_confirmed['Country/Region'] == 'China']

In [ ]:
for row in covid_19_confirmed_China.iterrows():
    name = row[1]['Province/State']
    assert(name in sheng_en_to_cn)

In [ ]:
from datetime import datetime
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import numpy as np
from shapely.geometry import Point
import mapclassify as mc
from matplotlib_scalebar.scalebar import ScaleBar

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'  
plt.rcParams['font.sans-serif'] = 'SimSun,Times New Roman'# 中文设置成宋体，除此之外的字体设置成New Roman  

# 定义CRS
albers_proj = '+proj=aea +lat_1=25 +lat_2=47 +lon_0=105'

def set_attatch_plot_range(ax):
    y1, y2 = ax.get_ylim()
    h = y2 - y1
    x1, x2 = ax.get_xlim()
    w = x2 - x1
    ax.set_ylim(y1, y1 + (h * 0.4))
    ax.set_xlim(x1 + (w * 0.55), x2 - (w * 0.05))

def plot_map_attach(fig, bs, map_with_covid_19, sample_date):
    # 副图框绘制
    ax = fig.add_axes([0.8, 0.18, 0.10, 0.15]) # left, bottom, width, height
    
    # 去除刻度
    ax.set_xticks([])
    ax.set_yticks([])
    
    map_with_covid_19.to_crs(albers_proj).plot( ax=ax,
                                                column=sample_date,
                                                cmap='Reds',
                                                scheme='UserDefined',
                                                classification_kwds={
                                                    'bins': bs
                                                },
                                                vmin=0,
                                                vmax=len(bs)-1,
                                                edgecolor='black',
                                                linewidth=0.2
                                               )

    map_jiuduanxian.geometry.to_crs(albers_proj).plot(ax=ax,
                                                      edgecolor='black',
                                                      linewidth=2,
                                                      alpha=0.5)
    
    set_attatch_plot_range(ax)

def set_plot_range(ax):
    y1, y2 = ax.get_ylim()
    h = y2 - y1
    ax.set_ylim(y2 - (h * 0.75), y2)
    
def plot_scalebar(ax):
    #scalebar = ScaleBar(dx=1*10**-3, units='km', length_fraction=0.1, height_fraction=0.001,
    scalebar = ScaleBar(dx=1, length_fraction=0.1, height_fraction=0.005,
                    font_properties={'family': 'Times New Roman', 'weight': 'normal', 'size': 12},
                    location='upper right', sep=1, frameon=False)
    
    ax.add_artist(scalebar)

def plot_compass(ax):
    x, y, arrow_length = 0.42, 0.09, 0.07
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=4, headwidth=7),
                ha='center', va='center', fontsize=10,
                xycoords=ax.transAxes)
    
def plot_map(ds, output_fig=True):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")

    df = covid_19_confirmed_China.loc[:, ['Province/State', sample_date]]
    df['cn_name'] = [sheng_en_to_cn[w[1]['Province/State']] for w in df.iterrows()]

    map_with_covid_19 = map_cn.set_index('OWNER').join(df.set_index('cn_name'))

    fig, ax = plt.subplots(figsize=(10, 10))

    bs = [0, 9, 49, 99, 999, 9999, 1e+10]
    ax = map_with_covid_19.to_crs(albers_proj).plot( ax=ax,
                                                     column=sample_date,
                                                     cmap='Reds',
                                                     legend=True,
                                                     scheme='UserDefined',
                                                     classification_kwds={
                                                         'bins': bs
                                                     },
                                                     legend_kwds={
                                                         'loc': 'lower left',
                                                         'title': '图例（单位：人）',
                                                         'shadow': True
                                                     },
                                                     vmin=0,
                                                     vmax=len(bs)-1,
                                                     edgecolor='black',
                                                     linewidth=0.2
                                                   )

    ax = map_jiuduanxian.geometry.to_crs(albers_proj).plot(ax=ax,
                                                           edgecolor='grey',
                                                           linewidth=3,
                                                           alpha=0.4)

    handles, labels = ax.get_legend_handles_labels()

    cmap = plt.get_cmap('Reds')

    # 得到mapclassify中的数据分层点
    bp = mc.UserDefined(map_with_covid_19[sample_date], bins=bs)
    bins = bp.bins

    labels = ['0'] + [f'{int(bins[_])+1} - {int(bins[_+1])}' for _ in range(0, len(bins) - 2)] + [f'≥{int(bins[-2])+1}']
    # 制作图例映射对象列表
    LegendElement = [ mpatches.Patch(facecolor=cmap(_/(len(labels)-1)), label=labels[_]) for _ in range(len(labels))]

    # 将制作好的图例映射对象列表导入legend()中，并配置相关参数
    ax.legend(handles = LegendElement,
              loc='lower left',
              fontsize=14,
              title='图例（单位：人）',
              title_fontsize=14,
              framealpha=0, # 去除边框
              borderpad=0.6)

    ax.axis('off')
    
    set_plot_range(ax)

    plot_map_attach(fig=fig,
                    bs=bs,
                    map_with_covid_19=map_with_covid_19,
                    sample_date=sample_date)
    
    plot_scalebar(ax)
    # plot_compass(ax)
    
    if output_fig:
        plt.savefig('c:/tmp/fig/China_{0}.png'.format(ds))

In [ ]:
plot_map('2020-1-22', True)

In [ ]:
plot_map('2020-2-22', True)

In [ ]:
plot_map('2020-3-22', True)

In [ ]:
plot_map('2020-4-22', True)

In [ ]:
plot_map('2020-5-22', True)

In [ ]:
plot_map('2020-6-22', True)

In [ ]:
plot_map('2020-7-22', True)

In [ ]:
plot_map('2020-8-22', True)

In [ ]:
plot_map('2020-9-22', True)